# Flatiron Health mBC: Final GBM build 

**OBJECTIVE: Build a gradient boosted survival model on the entirety of the Flation Health metastatic breast cancer data set.**

**BACKGROUND: For details on hyperparameter tuning of the gradient boosted model see notebook *Machine learning crude imputation*. Missingness will be crudely imputatated given similar test-set AUC performance when compared to MICE.** 

**OUTLINE:**
1. **Preprocessing**
2. **Gradient boosted model** 
3. **Calculate patient risk scores**
4. **Export final list of variables**

## 1. Preprocessing 

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

### Importing full dataframes

In [3]:
# Import training set and set PatientID as index.
train = pd.read_csv('train_full.csv')

In [4]:
row_ID(train)

(25341, 25341)

In [5]:
# Import test set and set PatientID as index.
test = pd.read_csv('test_full.csv')

In [6]:
row_ID(test)

(6336, 6336)

In [7]:
df = pd.concat([train, test])

In [8]:
row_ID(df)

(31677, 31677)

In [9]:
df = df.set_index('PatientID')

In [10]:
df.sample(3)

,gender,race,ethnicity,age,p_type,region,stage,met_year,delta_met_diagnosis,death_status,...,other_cancer,bone_met,thorax_met,lymph_met,liver_met,cns_met,skin_met,peritoneum_met,other_met,ses
PatientID,,,,,,,,,,,,,,,,,,,,,
FA9DE579150C9,F,White,Not Hispanic or Latino,75,COMMUNITY,south,III,2012,2943,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
FF8FD035684C3,F,White,Not Hispanic or Latino,75,COMMUNITY,midwest,III,2012,265,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
F5E62D086C8FC,F,White,hispanic_latino,50,ACADEMIC,unknown,III,2020,3706,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN


### Converting datatypes 

In [11]:
list(df.select_dtypes(include = ['object']).columns)

['gender',
 'race',
 'ethnicity',
 'p_type',
 'region',
 'stage',
 'ER',
 'HER2',
 'PR',
 'BRCA',
 'PIK3CA',
 'pdl1_n',
 'ecog_diagnosis']

In [12]:
to_be_categorical = list(df.select_dtypes(include = ['object']).columns)

In [13]:
to_be_categorical.append('met_year')

In [14]:
to_be_categorical.append('ses')

In [15]:
# Convert variables in list to categorical.
for x in list(to_be_categorical):
    df[x] = df[x].astype('category')

In [16]:
list(df.select_dtypes(include = ['category']).columns)

['gender',
 'race',
 'ethnicity',
 'p_type',
 'region',
 'stage',
 'met_year',
 'ER',
 'HER2',
 'PR',
 'BRCA',
 'PIK3CA',
 'pdl1_n',
 'ecog_diagnosis',
 'ses']

In [17]:
# Convert death_status into True or False (required for scikit-survival). 
df['death_status'] = df['death_status'].astype('bool')

### Dropping unneeded labs 

In [18]:
drop_labs = [
    'albumin_avg',
    'alp_avg',
    'alt_avg',
    'ast_avg',
    'bicarb_avg',
    'bun_avg',
    'calcium_avg',
    'chloride_avg',
    'creatinine_avg',
    'hemoglobin_avg',
    'neutrophil_count_avg',
    'platelet_avg',
    'potassium_avg',
    'sodium_avg',
    'total_bilirubin_avg',
    'wbc_avg',
    'albumin_max',
    'bicarb_max',
    'bun_max',
    'chloride_max',
    'hemoglobin_max',
    'neutrophil_count_max',
    'platelet_max',
    'potassium_max',
    'sodium_max',
    'alp_min',
    'alt_min',
    'ast_min',
    'bun_min',
    'calcium_min',
    'chloride_min',
    'creatinine_min',
    'neutrophil_count_min',
    'potassium_min',
    'total_bilirubin_min',
    'albumin_std',
    'alp_std',
    'alt_std',
    'ast_std',
    'bicarb_std',
    'bun_std',
    'calcium_std',
    'chloride_std',
    'creatinine_std',
    'hemoglobin_std',
    'neutrophil_count_std',
    'platelet_std',
    'potassium_std',
    'sodium_std',
    'total_bilirubin_std',
    'wbc_std',
    'albumin_slope',
    'alp_slope',
    'alt_slope',
    'ast_slope',
    'bicarb_slope',
    'bun_slope',
    'calcium_slope',
    'chloride_slope',
    'creatinine_slope',
    'hemoglobin_slope',
    'neutrophil_count_slope',
    'platelet_slope',
    'potassium_slope',
    'sodium_slope',
    'total_bilirubin_slope',
    'wbc_slope',
    'albumin_slope_na',
    'alp_slope_na',
    'alt_slope_na',
    'ast_slope_na',
    'bicarb_slope_na',
    'bun_slope_na',
    'calcium_slope_na',
    'chloride_slope_na',
    'creatinine_slope_na',
    'hemoglobin_slope_na',
    'neutrophil_count_slope_na',
    'platelet_slope_na',
    'potassium_slope_na',
    'sodium_slope_na',
    'total_bilirubin_slope_na',
    'wbc_slope_na']

In [19]:
df.shape

(31677, 216)

In [20]:
df = df.drop(columns = drop_labs)

In [21]:
df.shape

(31677, 133)

In [22]:
df.loc[:, 'alp_max_na'] = np.where(df['alp_max'].isna(), 1, 0)
df.loc[:, 'alt_max_na'] = np.where(df['alt_max'].isna(), 1, 0)
df.loc[:, 'ast_max_na'] = np.where(df['ast_max'].isna(), 1, 0)
df.loc[:, 'calcium_max_na'] = np.where(df['calcium_max'].isna(), 1, 0)
df.loc[:, 'creatinine_max_na'] = np.where(df['creatinine_max'].isna(), 1, 0)
df.loc[:, 'total_bilirubin_max_na'] = np.where(df['total_bilirubin_max'].isna(), 1, 0)
df.loc[:, 'wbc_max_na'] = np.where(df['wbc_max'].isna(), 1, 0)
df.loc[:, 'albumin_min_na'] = np.where(df['albumin_min'].isna(), 1, 0)
df.loc[:, 'bicarb_min_na'] = np.where(df['bicarb_min'].isna(), 1, 0)
df.loc[:, 'hemoglobin_min_na'] = np.where(df['hemoglobin_min'].isna(), 1, 0)
df.loc[:, 'platelet_min_na'] = np.where(df['platelet_min'].isna(), 1, 0)
df.loc[:, 'sodium_min_na'] = np.where(df['sodium_min'].isna(), 1, 0)
df.loc[:, 'wbc_min_na'] = np.where(df['wbc_min'].isna(), 1, 0)

In [23]:
df.shape

(31677, 146)

### Separate into X and Y 

In [24]:
# 'X' datasets
df_x = df.drop(columns = ['death_status', 'timerisk_activity']) #80% of data 

In [25]:
# 'Y' datasets
# Death status and time until event needs to be stored as a structured array to be compatible with scikit-survival
y_dtypes = df[['death_status', 'timerisk_activity']].dtypes

df_y = np.array([tuple(x) for x in df[['death_status', 'timerisk_activity']].values],
                dtype = list(zip(y_dtypes.index, y_dtypes)))

### Pipeline 

In [26]:
# List of numeric variables, excluding binary variables. 
numerical_features = [
    'age',
    'delta_met_diagnosis',
    'weight_diag',
    'bmi_diag',
    'weight_pct_change',
    'weight_slope',
    'albumin_diag',
    'alp_diag',
    'alt_diag',
    'ast_diag',
    'bicarb_diag',
    'bun_diag',
    'calcium_diag',
    'chloride_diag',
    'creatinine_diag',
    'hemoglobin_diag',
    'neutrophil_count_diag',
    'platelet_diag',
    'potassium_diag',
    'sodium_diag',
    'total_bilirubin_diag',
    'wbc_diag',
    'alp_max',
    'alt_max',
    'ast_max',
    'calcium_max',
    'creatinine_max',
    'total_bilirubin_max',
    'wbc_max',
    'albumin_min',
    'bicarb_min',
    'hemoglobin_min',
    'platelet_min',
    'sodium_min',
    'wbc_min',
    'icd_count']

# Transformer will impute column medians and then apply a standard scaler. 
numerical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('std_scaler', StandardScaler())])

In [27]:
# List of categorical features.
categorical_features = list(df_x.select_dtypes(include = ['category']).columns)

# One-hot-encode categorical features.
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

In [28]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder = 'passthrough')

## 2. Gradient boosted model 

In [29]:
from sklearn.pipeline import make_pipeline

from sksurv.ensemble import GradientBoostingSurvivalAnalysis

from joblib import dump, load 

In [30]:
df_x.shape

(31677, 144)

In [31]:
df_y.shape

(31677,)

In [32]:
gbm_final = make_pipeline(preprocessor, GradientBoostingSurvivalAnalysis(n_estimators = 750,
                                                                         learning_rate = 0.05,
                                                                         max_depth = 4,
                                                                         subsample = 0.75,
                                                                         verbose = 1,
                                                                         random_state = 42))

gbm_final.fit(df_x, df_y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      126432.7779          28.3984          219.53m
         2      126572.8783          27.1525          219.60m
         3      126770.8779          25.1774          219.51m
         4      126826.3604          23.4440          219.41m
         5      126349.7325          23.8042          219.09m
         6      126311.0702          21.5257          218.75m
         7      126330.9371          21.9645          218.41m
         8      126441.9730          20.0297          218.06m
         9      126161.2176          19.4870          217.80m
        10      126312.5995          18.6803          217.55m
        20      125343.9500          14.7005          213.87m
        30      124629.7805          10.5586          210.17m
        40      124441.5053           8.8807          207.47m
        50      124239.3364           7.2230          204.73m
        60      124498.4640           4.7499          202.03m
       

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'delta_met_diagnosis',
                                                   'weight_diag', 'bmi_diag',
                                                   'weight_pct_change',
                                                   'weight_slope',
                                                   'albumin_diag', 'alp_diag',
                                                   'alt_diag', 'ast_diag',
                                            

In [33]:
dump(gbm_final, 'gbm_final.joblib') 

['gbm_final.joblib']

## 3. Calculate patient risk scores 

In [34]:
# Calculate risk score for patients in training set. 
crude_risk_score_df = pd.DataFrame({'risk_score': gbm_final.predict(df_x)},
                                   index = df_x.index)

In [35]:
crude_risk_score_df.to_csv('crude_risk_score_df.csv', index = True, header = True)

## 4. Export final list of variables

In [36]:
import csv

In [37]:
breast_columns = list(df_x.columns)

In [38]:
with open('breast_columns.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(breast_columns)